In [3]:
# Instructions: glove.6B.100d.txt should be in this same folder
# If you are running for the first time, uncomment the below 2 lines
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove*.zip

In [4]:
%pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import ComplementNB
from sklearn.feature_extraction.text import TfidfVectorizer
import datetime

import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")

import os
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_filename = "glove.6B.100d.txt"
# If you are running for the first time, uncomment the below 2 lines
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove*.zip
glove_path = os.path.abspath(os.path.join(".", glove_filename))

word2vec_output_file = glove_filename + ".word2vec"

glove2word2vec(glove_path, word2vec_output_file)
word2vec_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zhizhouhuang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/var/folders/44/6b6tlgbd7z136wnf8vrdxjmw0000gn/T/ipykernel_66881/2908906817.py:28: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_path, word2vec_output_file)


In [6]:
csv.field_size_limit(999999)
train = pd.read_csv("../raw_data/fulltrain.csv", header=None, names=["class", "text"])
X_train = train["text"]
y_train = train["class"]


test = pd.read_csv("../raw_data/balancedtest.csv", header=None, names=["class", "text"])
X_test = test["text"]
y_test = test["class"]

In [7]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

label_encoder = LabelEncoder()
one_hot_encoder = OneHotEncoder()
y_train_labeled = one_hot_encoder.fit_transform(
    np.array(y_train).reshape(-1, 1)
).toarray()
y_test_labeled = one_hot_encoder.transform(np.array(y_test).reshape(-1, 1)).toarray()

In [8]:
from tqdm import tqdm


class Word2VecVectorizer:
    def __init__(self, model):
        print("Loading in word vectors...")
        self.word_vectors = model
        print("Finished loading in word vectors")

    def fit(self, data):
        pass

    def transform(self, data):
        # determine the dimensionality of vectors
        v = self.word_vectors.get_vector("queen")
        self.D = v.shape[0]

        X = np.zeros((len(data), self.D * 3))
        n = 0
        emptycount = 0
        for sentence in tqdm(data):
            tokens = nltk.word_tokenize(sentence.lower())
            vecs = []
            m = 0
            for word in tokens:
                try:
                    # throws KeyError if word not found
                    vec = self.word_vectors.get_vector(word)
                    vecs.append(vec)
                    m += 1
                except KeyError:
                    pass
            if len(vecs) > 0:
                vecs = np.array(vecs)
                X[n] = np.concatenate(
                    (vecs.mean(axis=0), vecs.min(axis=0), vecs.max(axis=0))
                )
            else:
                emptycount += 1
            n += 1
        print(
            "Number of samples with no words found: %s / %s" % (emptycount, len(data))
        )
        return X

    def fit_transform(self, data):
        self.fit(data)
        return self.transform(data)

In [9]:
word2vec_vectorizer = Word2VecVectorizer(word2vec_model)
X_train_word2vec = word2vec_vectorizer.fit_transform(X_train)

Loading in word vectors...
Finished loading in word vectors


100%|██████████| 48854/48854 [01:42<00:00, 477.97it/s] 

Number of samples with no words found: 2 / 48854


In [10]:
# apply to test
X_test_word2vec = word2vec_vectorizer.fit_transform(X_test)

100%|██████████| 3000/3000 [00:05<00:00, 510.30it/s]

Number of samples with no words found: 0 / 3000


In [11]:
print(X_train_word2vec.shape)

(48854, 300)


In [12]:
from keras import layers

max_length = 1000
vectorizer = layers.TextVectorization(
    max_tokens=20000, output_sequence_length=max_length
)
vectorizer.adapt(X_train)

In [13]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

X_train_vector = vectorizer(X_train)
X_test_vector = vectorizer(X_test)

print(X_train_vector.shape)
print(X_train_vector[:2])

(48854, 1000)
tf.Tensor(
[[   6  226  310 ...    0    0    0]
 [   2 3946    4 ...    0    0    0]], shape=(2, 1000), dtype=int64)


In [14]:
embedding_dim = 100

glove_filename = "glove.6B." + str(embedding_dim) + "d.txt"
glove_path = os.path.abspath(os.path.join(".", glove_filename))

embeddings_index = {}
with open(glove_path) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [15]:
num_tokens = len(voc) + 2
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 19415 words (585 misses)


In [13]:
print(embedding_matrix.shape)

(20002, 100)


In [14]:
!pip3 install tensorflow-addons


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [16]:
import keras
from keras.layers import (
    Flatten,
    Embedding,
    GlobalAveragePooling1D,
    Bidirectional,
    LSTM,
    GlobalAveragePooling2D,
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, Adagrad
import tensorflow as tf

max_features = 100
epochs = 10
inputs = keras.Input(shape=(None,), dtype="int32")
embedding_layer = Embedding(num_tokens, embedding_dim, input_length=max_length)
embedding_layer.build()
embedding_layer.set_weights([embedding_matrix])
x = embedding_layer(inputs)

# Adding Bidirectional LSTM layers
x = LSTM(64)(x)
# x = Bidirectional(LSTM(64, return_sequences=True))(x)
# x = Bidirectional(LSTM(64))(x)  # Adding Flatten layer to convert output to 2D tensor
# x = Flatten()(x)
# x = Dense(64, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)


model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.001),
    metrics=[
        "accuracy",
        tf.keras.metrics.F1Score(
            average="macro", threshold=None, name="f1_score", dtype=None
        ),
    ],
)
model.summary()

checkpoint_filepath = "checkpoint_model_lstm_64.keras"

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_f1_score",
    mode="max",
    save_best_only=True,
)

model.fit(
    X_train_vector,
    y_train_labeled,
    epochs=epochs,
    validation_data=(X_test_vector, y_test_labeled),
    callbacks=[model_checkpoint_callback],
)

model.load_weights(checkpoint_filepath)
y_pred = model.predict(X_test_vector)

y_pred_inverse = one_hot_encoder.inverse_transform(y_pred)

print(classification_report(y_test, y_pred_inverse))

/Users/zhizhouhuang/Desktop/Labeled-Unreliable-News/venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 100)      │     2,000,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        42,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,044,652 (7.80 MB)

 Trainable params: 2,044,652 (7.80 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 488s 318ms/step - accuracy: 0.4001 - f1_score: 0.2545 - loss: 1.2451 - val_accuracy: 0.2803 - val_f1_score: 0.1534 - val_loss: 1.4864
Epoch 2/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 470s 308ms/step - accuracy: 0.4348 - f1_score: 0.2674 - loss: 1.1977 - val_accuracy: 0.4473 - val_f1_score: 0.3244 - val_loss: 1.3914
Epoch 3/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 495s 324ms/step - accuracy: 0.5637 - f1_score: 0.3715 - loss: 1.0362 - val_accuracy: 0.4653 - val_f1_score: 0.3401 - val_loss: 1.4503
Epoch 4/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 530s 347ms/step - accuracy: 0.6736 - f1_score: 0.5105 - loss: 0.7329 - val_accuracy: 0.4520 - val_f1_score: 0.3896 - val_loss: 2.3075
Epoch 5/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 485s 318ms/step - accuracy: 0.7773 - f1_score: 0.6813 - loss: 0.4891 - val_accuracy: 0.5250 - val_f1_score: 0.4779 - val_loss: 1.8568
Epoch 6/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 519s 340ms/step - accuracy: 0.8102 - f1_score: 0.7531 - loss: 0.3892 - val_acc

In [18]:
import keras
from keras.layers import (
    Flatten,
    Embedding,
    GlobalAveragePooling1D,
    Bidirectional,
    LSTM,
    GlobalAveragePooling2D,
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, Adagrad
import tensorflow as tf

max_features = 100
epochs = 10
inputs = keras.Input(shape=(None,), dtype="int32")
embedding_layer = Embedding(num_tokens, embedding_dim, input_length=max_length)
embedding_layer.build()
embedding_layer.set_weights([embedding_matrix])
x = embedding_layer(inputs)

# Adding Bidirectional LSTM layers
x = Bidirectional(LSTM(64, return_sequences=True))(x)
x = Bidirectional(LSTM(64))(x)
# x = Dense(64, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)


model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.001),
    metrics=[
        "accuracy",
        tf.keras.metrics.F1Score(
            average="macro", threshold=None, name="f1_score", dtype=None
        ),
    ],
)
model.summary()

checkpoint_filepath = "checkpoint_model_2_bi_lstm_64.keras"

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_f1_score",
    mode="max",
    save_best_only=True,
)

model.fit(
    X_train_vector,
    y_train_labeled,
    epochs=epochs,
    validation_data=(X_test_vector, y_test_labeled),
    callbacks=[model_checkpoint_callback],
)

model.load_weights(checkpoint_filepath)
y_pred = model.predict(X_test_vector)

y_pred_inverse = one_hot_encoder.inverse_transform(y_pred)

print(classification_report(y_test, y_pred_inverse))

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_2 (Embedding)         │ (None, None, 100)      │     2,000,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, None, 128)      │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,187,756 (8.35 MB)

 Trainable params: 2,187,756 (8.35 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 1365s 892ms/step - accuracy: 0.7544 - f1_score: 0.7383 - loss: 0.6312 - val_accuracy: 0.5920 - val_f1_score: 0.5631 - val_loss: 1.4156
Epoch 2/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 1410s 923ms/step - accuracy: 0.9652 - f1_score: 0.9620 - loss: 0.1211 - val_accuracy: 0.5860 - val_f1_score: 0.5731 - val_loss: 1.3938
Epoch 3/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 1572s 1s/step - accuracy: 0.9802 - f1_score: 0.9784 - loss: 0.0666 - val_accuracy: 0.6187 - val_f1_score: 0.6025 - val_loss: 1.9994
Epoch 4/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 1437s 941ms/step - accuracy: 0.9926 - f1_score: 0.9918 - loss: 0.0300 - val_accuracy: 0.6150 - val_f1_score: 0.5947 - val_loss: 3.0107
Epoch 5/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 1335s 874ms/step - accuracy: 0.9948 - f1_score: 0.9943 - loss: 0.0206 - val_accuracy: 0.6497 - val_f1_score: 0.6315 - val_loss: 2.4458
Epoch 6/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 1452s 951ms/step - accuracy: 0.9962 - f1_score: 0.9958 - loss: 0.0168 - val_

In [21]:
import keras
from keras.layers import (
    Flatten,
    Embedding,
    GlobalAveragePooling1D,
    Bidirectional,
    LSTM,
    GlobalAveragePooling2D,
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, Adagrad
import tensorflow as tf

max_features = 100
epochs = 10
inputs = keras.Input(shape=(None,), dtype="int32")
embedding_layer = Embedding(num_tokens, embedding_dim, input_length=max_length)
embedding_layer.build()
embedding_layer.set_weights([embedding_matrix])
x = embedding_layer(inputs)
# Adding Bidirectional LSTM layers
x = Bidirectional(LSTM(64, return_sequences=True))(x)
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(64))(x)
# x = Dense(64, activation="relu")(x)
# x = Dropout(0.5)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)


model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.001),
    metrics=[
        "accuracy",
        tf.keras.metrics.F1Score(
            average="macro", threshold=None, name="f1_score", dtype=None
        ),
    ],
)
model.summary()

checkpoint_filepath = "checkpoint_model_bi_lstm_64.keras"

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_f1_score",
    mode="max",
    save_best_only=True,
)

model.fit(
    X_train_vector,
    y_train_labeled,
    epochs=epochs,
    validation_data=(X_test_vector, y_test_labeled),
    callbacks=[model_checkpoint_callback],
)

model.load_weights(checkpoint_filepath)
y_pred = model.predict(X_test_vector)

y_pred_inverse = one_hot_encoder.inverse_transform(y_pred)

print(classification_report(y_test, y_pred_inverse))

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_5 (Embedding)         │ (None, None, 100)      │     2,000,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_6 (Bidirectional) │ (None, 128)            │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,085,196 (7.95 MB)

 Trainable params: 2,085,196 (7.95 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 764s 499ms/step - accuracy: 0.8201 - f1_score: 0.8004 - loss: 0.4827 - val_accuracy: 0.5890 - val_f1_score: 0.5661 - val_loss: 1.3082
Epoch 2/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 759s 497ms/step - accuracy: 0.9763 - f1_score: 0.9743 - loss: 0.0705 - val_accuracy: 0.6187 - val_f1_score: 0.6033 - val_loss: 1.5458
Epoch 3/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 4999s 3s/step - accuracy: 0.9925 - f1_score: 0.9920 - loss: 0.0243 - val_accuracy: 0.6103 - val_f1_score: 0.5979 - val_loss: 1.7963
Epoch 4/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 769s 504ms/step - accuracy: 0.9969 - f1_score: 0.9967 - loss: 0.0116 - val_accuracy: 0.6213 - val_f1_score: 0.5888 - val_loss: 1.9525
Epoch 5/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 762s 499ms/step - accuracy: 0.9986 - f1_score: 0.9985 - loss: 0.0051 - val_accuracy: 0.6003 - val_f1_score: 0.5911 - val_loss: 2.0958
Epoch 6/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 770s 505ms/step - accuracy: 0.9990 - f1_score: 0.9990 - loss: 0.0032 - val_accur

In [24]:
import keras
from keras.layers import (
    Flatten,
    Embedding,
    GlobalAveragePooling1D,
    Bidirectional,
    LSTM,
    GlobalAveragePooling2D,
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, Adagrad
import tensorflow as tf

max_features = 100
epochs = 10
print(max_length)
inputs = keras.Input(shape=(None,), dtype="int32")
embedding_layer = Embedding(num_tokens, embedding_dim, input_length=max_length)
embedding_layer.build()
embedding_layer.set_weights([embedding_matrix])
x = embedding_layer(inputs)

# Adding Bidirectional LSTM layers
# x = Bidirectional(LSTM(64, return_sequences=True))(x)
x = Bidirectional(LSTM(64))(x)
# x = Dense(64, activation="relu")(x)
# x = Dropout(0.5)(x)
# x = Dense(32, activation="relu")(x)
# x = Dropout(0.5)(x)
outputs = Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)


model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.001),
    metrics=[
        "accuracy",
        tf.keras.metrics.F1Score(
            average="macro", threshold=None, name="f1_score", dtype=None
        ),
    ],
)
model.summary()
model.load_weights(checkpoint_filepath)

checkpoint_filepath = "checkpoint_model_bi_lstm_64.keras"

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_f1_score",
    mode="max",
    save_best_only=True,
)

model.fit(
    X_train_vector,
    y_train_labeled,
    epochs=2,
    validation_data=(X_test_vector, y_test_labeled),
    callbacks=[model_checkpoint_callback],
)

model.load_weights(checkpoint_filepath)
y_pred = model.predict(X_test_vector)

y_pred_inverse = one_hot_encoder.inverse_transform(y_pred)

print(classification_report(y_test, y_pred_inverse))

1000


/Users/zhizhouhuang/Desktop/Labeled-Unreliable-News/venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_7 (Embedding)         │ (None, None, 100)      │     2,000,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_8 (Bidirectional) │ (None, 128)            │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,085,196 (7.95 MB)

 Trainable params: 2,085,196 (7.95 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/2


/Users/zhizhouhuang/Desktop/Labeled-Unreliable-News/venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 20 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


1527/1527 ━━━━━━━━━━━━━━━━━━━━ 757s 495ms/step - accuracy: 0.9995 - f1_score: 0.9995 - loss: 0.0014 - val_accuracy: 0.6327 - val_f1_score: 0.6214 - val_loss: 2.4057
Epoch 2/2
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 764s 500ms/step - accuracy: 0.9999 - f1_score: 0.9999 - loss: 3.0617e-04 - val_accuracy: 0.6317 - val_f1_score: 0.6139 - val_loss: 2.5912
94/94 ━━━━━━━━━━━━━━━━━━━━ 8s 84ms/step
              precision    recall  f1-score   support

           1       0.86      0.77      0.82       750
           2       0.72      0.45      0.56       750
           3       0.46      0.38      0.42       750
           4       0.56      0.93      0.70       750

    accuracy                           0.63      3000
   macro avg       0.65      0.63      0.62      3000
weighted avg       0.65      0.63      0.62      3000



In [27]:
import keras
from keras.layers import (
    Flatten,
    Embedding,
    GlobalAveragePooling1D,
    Bidirectional,
    LSTM,
    GlobalAveragePooling2D,
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, Adagrad
import tensorflow as tf

max_features = 100
epochs = 10
inputs = keras.Input(shape=(None,), dtype="int32")
embedding_layer = Embedding(num_tokens, embedding_dim, input_length=max_length)
embedding_layer.build()
embedding_layer.set_weights([embedding_matrix])
x = embedding_layer(inputs)
# Adding Bidirectional LSTM layers
x = Bidirectional(LSTM(32, return_sequences=True))(x)
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(32))(x)
# x = Dense(64, activation="relu")(x)
# x = Dropout(0.5)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)


model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.001),
    metrics=[
        "accuracy",
        tf.keras.metrics.F1Score(
            average="macro", threshold=None, name="f1_score", dtype=None
        ),
    ],
)
model.summary()

checkpoint_filepath = "checkpoint_model_bi_lstm_64_2.keras"

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_f1_score",
    mode="max",
    save_best_only=True,
)

model.fit(
    X_train_vector,
    y_train_labeled,
    epochs=epochs,
    validation_data=(X_test_vector, y_test_labeled),
    callbacks=[model_checkpoint_callback],
)

model.load_weights(checkpoint_filepath)
y_pred = model.predict(X_test_vector)

y_pred_inverse = one_hot_encoder.inverse_transform(y_pred)

print(classification_report(y_test, y_pred_inverse))

Model: "functional_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)     │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_10 (Embedding)        │ (None, None, 100)      │     2,000,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_13                │ (None, None, 64)       │        34,048 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, None, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_14                │ (None, 64)             │        24,832 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,061,292 (7.86 MB)

 Trainable params: 2,061,292 (7.86 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 696s 453ms/step - accuracy: 0.7535 - f1_score: 0.7320 - loss: 0.6275 - val_accuracy: 0.5893 - val_f1_score: 0.5727 - val_loss: 1.6249
Epoch 2/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 695s 455ms/step - accuracy: 0.9603 - f1_score: 0.9569 - loss: 0.1339 - val_accuracy: 0.5920 - val_f1_score: 0.5788 - val_loss: 1.5544
Epoch 3/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 693s 454ms/step - accuracy: 0.9833 - f1_score: 0.9819 - loss: 0.0619 - val_accuracy: 0.5637 - val_f1_score: 0.5523 - val_loss: 2.1609
Epoch 4/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 689s 451ms/step - accuracy: 0.9911 - f1_score: 0.9904 - loss: 0.0367 - val_accuracy: 0.6253 - val_f1_score: 0.6178 - val_loss: 2.2977
Epoch 5/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 703s 461ms/step - accuracy: 0.9949 - f1_score: 0.9945 - loss: 0.0207 - val_accuracy: 0.6283 - val_f1_score: 0.6171 - val_loss: 2.5565
Epoch 6/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 709s 465ms/step - accuracy: 0.9960 - f1_score: 0.9957 - loss: 0.0159 - val_acc

In [17]:
import keras
from keras.layers import (
    Flatten,
    Embedding,
    GlobalAveragePooling1D,
    Bidirectional,
    LSTM,
    GlobalAveragePooling2D,
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, Adagrad
import tensorflow as tf

max_features = 100
epochs = 10
inputs = keras.Input(shape=(None,), dtype="int32")
embedding_layer = Embedding(num_tokens, embedding_dim, input_length=max_length)
embedding_layer.build()
embedding_layer.set_weights([embedding_matrix])
x = embedding_layer(inputs)

x = LSTM(64, return_sequences=True)(x)  # To add pooling here
x = GlobalAveragePooling1D()(x)  # Pooling layer
x = Dropout(0.5)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)


model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.001),
    metrics=[
        "accuracy",
        tf.keras.metrics.F1Score(
            average="macro", threshold=None, name="f1_score", dtype=None
        ),
    ],
)
model.summary()

checkpoint_filepath = "checkpoint_model_lstm_64_pooled.keras"

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_f1_score",
    mode="max",
    save_best_only=True,
)

model.fit(
    X_train_vector,
    y_train_labeled,
    epochs=epochs,
    validation_data=(X_test_vector, y_test_labeled),
    callbacks=[model_checkpoint_callback],
)

model.load_weights(checkpoint_filepath)
y_pred = model.predict(X_test_vector)

y_pred_inverse = one_hot_encoder.inverse_transform(y_pred)

print(classification_report(y_test, y_pred_inverse))

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, None, 100)      │     2,000,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, None, 64)       │        42,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,044,652 (7.80 MB)

 Trainable params: 2,044,652 (7.80 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 537s 351ms/step - accuracy: 0.5935 - f1_score: 0.5079 - loss: 0.9464 - val_accuracy: 0.6877 - val_f1_score: 0.6709 - val_loss: 1.0178
Epoch 2/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 577s 378ms/step - accuracy: 0.9004 - f1_score: 0.8927 - loss: 0.3045 - val_accuracy: 0.7450 - val_f1_score: 0.7341 - val_loss: 1.3746
Epoch 3/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 580s 380ms/step - accuracy: 0.9608 - f1_score: 0.9578 - loss: 0.1384 - val_accuracy: 0.7553 - val_f1_score: 0.7467 - val_loss: 1.4207
Epoch 4/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 2164s 1s/step - accuracy: 0.9784 - f1_score: 0.9769 - loss: 0.0783 - val_accuracy: 0.7453 - val_f1_score: 0.7378 - val_loss: 2.1318
Epoch 5/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 506s 331ms/step - accuracy: 0.9857 - f1_score: 0.9847 - loss: 0.0527 - val_accuracy: 0.7327 - val_f1_score: 0.7231 - val_loss: 1.5267
Epoch 6/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 518s 339ms/step - accuracy: 0.9922 - f1_score: 0.9916 - loss: 0.0320 - val_accur

In [20]:
import keras
from keras.layers import (
    Flatten,
    Embedding,
    GlobalAveragePooling1D,
    Bidirectional,
    LSTM,
    GlobalAveragePooling2D,
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, Adagrad
import tensorflow as tf

max_features = 100
epochs = 10
inputs = keras.Input(shape=(None,), dtype="int32")
embedding_layer = Embedding(num_tokens, embedding_dim, input_length=max_length)
embedding_layer.build()
embedding_layer.set_weights([embedding_matrix])
x = embedding_layer(inputs)

x = GlobalAveragePooling1D()(x)  # Pooling layer

x = keras.layers.Reshape((1, -1))(x)
x = LSTM(64)(x)  # To add pooling here
x = Dropout(0.5)(x)
x = Dense(64, activation="sigmoid")(x)
x = Dropout(0.5)(x)
x = Dense(32, activation="sigmoid")(x)
x = Dropout(0.5)(x)
outputs = Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)


model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.001),
    metrics=[
        "accuracy",
        tf.keras.metrics.F1Score(
            average="macro", threshold=None, name="f1_score", dtype=None
        ),
    ],
)
model.summary()

checkpoint_filepath = "checkpoint_model_pool_lstm_64_dense.keras"
model.load_weights(checkpoint_filepath)

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_f1_score",
    mode="max",
    save_best_only=True,
)

model.fit(
    X_train_vector,
    y_train_labeled,
    epochs=epochs,
    validation_data=(X_test_vector, y_test_labeled),
    callbacks=[model_checkpoint_callback],
)

model.load_weights(checkpoint_filepath)
y_pred = model.predict(X_test_vector)

y_pred_inverse = one_hot_encoder.inverse_transform(y_pred)

print(classification_report(y_test, y_pred_inverse))

/Users/zhizhouhuang/Desktop/Labeled-Unreliable-News/venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_4 (Embedding)         │ (None, None, 100)      │     2,000,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_4      │ (None, 100)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_2 (Reshape)             │ (None, 1, 100)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 64)             │        42,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,048,812 (7.82 MB)

 Trainable params: 2,048,812 (7.82 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


/Users/zhizhouhuang/Desktop/Labeled-Unreliable-News/venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


1527/1527 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - accuracy: 0.9293 - f1_score: 0.9254 - loss: 0.2399 - val_accuracy: 0.6750 - val_f1_score: 0.6633 - val_loss: 1.7933
Epoch 2/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9346 - f1_score: 0.9307 - loss: 0.2258 - val_accuracy: 0.7113 - val_f1_score: 0.6971 - val_loss: 1.6257
Epoch 3/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9310 - f1_score: 0.9267 - loss: 0.2333 - val_accuracy: 0.7060 - val_f1_score: 0.6777 - val_loss: 1.6212
Epoch 4/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.9417 - f1_score: 0.9386 - loss: 0.2023 - val_accuracy: 0.6413 - val_f1_score: 0.6331 - val_loss: 1.9510
Epoch 5/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.9397 - f1_score: 0.9367 - loss: 0.2033 - val_accuracy: 0.7267 - val_f1_score: 0.7073 - val_loss: 1.6144
Epoch 6/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9370 - f1_score: 0.9335 - loss: 0.2143 - val_accuracy: 0.6553 - val_f1_

In [22]:
import keras
from keras.layers import (
    Flatten,
    Embedding,
    GlobalAveragePooling1D,
    Bidirectional,
    LSTM,
    GlobalAveragePooling2D,
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, Adagrad
import tensorflow as tf

max_features = 100
epochs = 10
inputs = keras.Input(shape=(None,), dtype="int32")
embedding_layer = Embedding(num_tokens, embedding_dim, input_length=max_length)
embedding_layer.build()
embedding_layer.set_weights([embedding_matrix])
x = embedding_layer(inputs)

x = GlobalAveragePooling1D()(x)  # Pooling layer

x = keras.layers.Reshape((1, -1))(x)
x = LSTM(128)(x)  # To add pooling here
x = Dropout(0.5)(x)
x = Dense(64, activation="sigmoid")(x)
x = Dropout(0.5)(x)
x = Dense(32, activation="sigmoid")(x)
x = Dropout(0.5)(x)
outputs = Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)


model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.001),
    metrics=[
        "accuracy",
        tf.keras.metrics.F1Score(
            average="macro", threshold=None, name="f1_score", dtype=None
        ),
    ],
)
model.summary()

checkpoint_filepath = "checkpoint_model_pool_lstm_64_dense.keras"

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_f1_score",
    mode="max",
    save_best_only=True,
)

model.fit(
    X_train_vector,
    y_train_labeled,
    epochs=epochs,
    validation_data=(X_test_vector, y_test_labeled),
    callbacks=[model_checkpoint_callback],
)

model.load_weights(checkpoint_filepath)
y_pred = model.predict(X_test_vector)

y_pred_inverse = one_hot_encoder.inverse_transform(y_pred)

print(classification_report(y_test, y_pred_inverse))

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_6 (Embedding)         │ (None, None, 100)      │     2,000,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_6      │ (None, 100)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_4 (Reshape)             │ (None, 1, 100)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 128)            │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,127,916 (8.12 MB)

 Trainable params: 2,127,916 (8.12 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - accuracy: 0.4230 - f1_score: 0.3131 - loss: 1.2438 - val_accuracy: 0.5140 - val_f1_score: 0.4249 - val_loss: 1.4380
Epoch 2/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.6593 - f1_score: 0.5740 - loss: 0.8013 - val_accuracy: 0.6803 - val_f1_score: 0.6558 - val_loss: 1.1659
Epoch 3/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.8426 - f1_score: 0.8252 - loss: 0.4708 - val_accuracy: 0.6783 - val_f1_score: 0.6744 - val_loss: 1.1438
Epoch 4/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.8849 - f1_score: 0.8750 - loss: 0.3739 - val_accuracy: 0.6983 - val_f1_score: 0.6814 - val_loss: 1.1641
Epoch 5/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.9053 - f1_score: 0.8989 - loss: 0.3103 - val_accuracy: 0.6847 - val_f1_score: 0.6630 - val_loss: 1.2515
Epoch 6/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.9072 - f1_score: 0.9009 - loss: 0.2989 - val_accuracy: 0.705

In [25]:
import keras
from keras.layers import (
    Flatten,
    Embedding,
    GlobalAveragePooling1D,
    Bidirectional,
    LSTM,
    GlobalAveragePooling2D,
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, Adagrad
import tensorflow as tf

max_features = 100
epochs = 10
inputs = keras.Input(shape=(None,), dtype="int32")
embedding_layer = Embedding(num_tokens, embedding_dim, input_length=max_length)
embedding_layer.build()
embedding_layer.set_weights([embedding_matrix])
x = embedding_layer(inputs)

x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
# x = layers.Bidirectional(layers.LSTM(64, return ))(x)
x = GlobalAveragePooling1D()(x)  # Pooling layer
x = Dropout(0.5)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)


model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.001),
    metrics=[
        "accuracy",
        tf.keras.metrics.F1Score(
            average="macro", threshold=None, name="f1_score", dtype=None
        ),
    ],
)
model.summary()

checkpoint_filepath = "checkpoint_model_bi-lstm_64_pooled.keras"

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_f1_score",
    mode="max",
    save_best_only=True,
)

model.fit(
    X_train_vector,
    y_train_labeled,
    epochs=epochs,
    validation_data=(X_test_vector, y_test_labeled),
    callbacks=[model_checkpoint_callback],
)

model.load_weights(checkpoint_filepath)
y_pred = model.predict(X_test_vector)

y_pred_inverse = one_hot_encoder.inverse_transform(y_pred)

print(classification_report(y_test, y_pred_inverse))

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)     │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_9 (Embedding)         │ (None, None, 100)      │     2,000,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, None, 128)      │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_9      │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,088,940 (7.97 MB)

 Trainable params: 2,088,940 (7.97 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 535s 349ms/step - accuracy: 0.6046 - f1_score: 0.5349 - loss: 0.9330 - val_accuracy: 0.7407 - val_f1_score: 0.7308 - val_loss: 1.0729
Epoch 2/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 542s 355ms/step - accuracy: 0.9065 - f1_score: 0.9021 - loss: 0.3005 - val_accuracy: 0.7723 - val_f1_score: 0.7671 - val_loss: 1.1154
Epoch 3/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 548s 359ms/step - accuracy: 0.9574 - f1_score: 0.9558 - loss: 0.1514 - val_accuracy: 0.7100 - val_f1_score: 0.6801 - val_loss: 2.3020
Epoch 4/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 543s 355ms/step - accuracy: 0.9773 - f1_score: 0.9760 - loss: 0.0843 - val_accuracy: 0.7703 - val_f1_score: 0.7683 - val_loss: 1.5905
Epoch 5/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 542s 355ms/step - accuracy: 0.9865 - f1_score: 0.9856 - loss: 0.0514 - val_accuracy: 0.7380 - val_f1_score: 0.7358 - val_loss: 1.9489
Epoch 6/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 534s 350ms/step - accuracy: 0.9899 - f1_score: 0.9893 - loss: 0.0394 - val_acc

In [26]:
import keras
from keras.layers import (
    Flatten,
    Embedding,
    GlobalAveragePooling1D,
    SimpleRNN,  # Changed from LSTM to SimpleRNN
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
import tensorflow as tf

max_features = 100
epochs = 10
inputs = keras.Input(shape=(None,), dtype="int32")
embedding_layer = Embedding(num_tokens, embedding_dim, input_length=max_length)
embedding_layer.build()
embedding_layer.set_weights([embedding_matrix])
x = embedding_layer(inputs)

x = SimpleRNN(64, return_sequences=True)(x)
x = GlobalAveragePooling1D()(x)  # Pooling layer
x = Dropout(0.5)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.001),
    metrics=[
        "accuracy",
        tf.keras.metrics.F1Score(
            average="macro", threshold=None, name="f1_score", dtype=None
        ),
    ],
)
model.summary()

checkpoint_filepath = "checkpoint_model_rnn_64_pooled.keras"

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_f1_score",
    mode="max",
    save_best_only=True,
)

model.fit(
    X_train_vector,
    y_train_labeled,
    epochs=epochs,
    validation_data=(X_test_vector, y_test_labeled),
    callbacks=[model_checkpoint_callback],
)

model.load_weights(checkpoint_filepath)
y_pred = model.predict(X_test_vector)

y_pred_inverse = one_hot_encoder.inverse_transform(y_pred)

print(classification_report(y_test, y_pred_inverse))

/Users/zhizhouhuang/Desktop/Labeled-Unreliable-News/venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_11 (InputLayer)     │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_10 (Embedding)        │ (None, None, 100)      │     2,000,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, None, 64)       │        10,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_10     │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,012,972 (7.68 MB)

 Trainable params: 2,012,972 (7.68 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 203s 132ms/step - accuracy: 0.4752 - f1_score: 0.3764 - loss: 1.1378 - val_accuracy: 0.4910 - val_f1_score: 0.3506 - val_loss: 1.3014
Epoch 2/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 202s 132ms/step - accuracy: 0.5773 - f1_score: 0.4852 - loss: 0.9703 - val_accuracy: 0.5970 - val_f1_score: 0.5711 - val_loss: 1.4805
Epoch 3/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 203s 133ms/step - accuracy: 0.7449 - f1_score: 0.7214 - loss: 0.6448 - val_accuracy: 0.6977 - val_f1_score: 0.6890 - val_loss: 1.1510
Epoch 4/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 204s 133ms/step - accuracy: 0.7815 - f1_score: 0.7663 - loss: 0.6956 - val_accuracy: 0.6743 - val_f1_score: 0.6543 - val_loss: 1.4193
Epoch 5/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 200s 131ms/step - accuracy: 0.8740 - f1_score: 0.8664 - loss: 0.3744 - val_accuracy: 0.6987 - val_f1_score: 0.6854 - val_loss: 1.2281
Epoch 6/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 205s 134ms/step - accuracy: 0.8921 - f1_score: 0.8857 - loss: 0.3196 - val_acc

In [30]:
y_pred[:5]


array([[1.0080810e-01, 5.1619965e-01, 3.2853299e-01, 5.4459173e-02],
       [9.7026002e-01, 2.0893443e-05, 3.3073637e-03, 2.6411740e-02],
       [9.2635077e-01, 1.1785535e-06, 2.8709776e-03, 7.0777044e-02],
       [1.4434965e-01, 8.8140194e-05, 6.4813770e-02, 7.9074854e-01],
       [4.6552014e-01, 3.1910229e-02, 1.6685601e-01, 3.3571362e-01]],
      dtype=float32)

In [37]:
test = pd.read_csv("../raw_data/balancedtest.csv", header=None, names=["class", "text"])

In [38]:
print(test[:5])

# append the predictions to the test data
test["rnn_predicted"] = y_pred_inverse

print(test[:5])

   class                                               text
0      1  When so many actors seem content to churn out ...
1      1   In what football insiders are calling an unex...
2      1  In a freak accident following Game 3 of the N....
3      1  North Koreas official news agency announced to...
4      1  The former Alaska Governor Sarah Palin would b...
   class                                               text  rnn_predicted
0      1  When so many actors seem content to churn out ...              2
1      1   In what football insiders are calling an unex...              1
2      1  In a freak accident following Game 3 of the N....              1
3      1  North Koreas official news agency announced to...              4
4      1  The former Alaska Governor Sarah Palin would b...              1


In [35]:
import keras
from keras.layers import (
    Flatten,
    Embedding,
    GlobalAveragePooling1D,
    Bidirectional,
    LSTM,
    GlobalAveragePooling2D,
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, Adagrad
import tensorflow as tf

max_features = 100
epochs = 10
inputs = keras.Input(shape=(None,), dtype="int32")
embedding_layer = Embedding(num_tokens, embedding_dim, input_length=max_length)
embedding_layer.build()
embedding_layer.set_weights([embedding_matrix])
x = embedding_layer(inputs)

x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
# x = layers.Bidirectional(layers.LSTM(64, return ))(x)
x = GlobalAveragePooling1D()(x)  # Pooling layer
x = Dropout(0.5)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)


model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.001),
    metrics=[
        "accuracy",
        tf.keras.metrics.F1Score(
            average="macro", threshold=None, name="f1_score", dtype=None
        ),
    ],
)
model.summary()

checkpoint_filepath = "checkpoint_model_bi-lstm_64_pooled.keras"

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_f1_score",
    mode="max",
    save_best_only=True,
)

model.fit(
    X_train_vector,
    y_train_labeled,
    epochs=epochs,
    validation_data=(X_test_vector, y_test_labeled),
    callbacks=[model_checkpoint_callback],
)

model.load_weights(checkpoint_filepath)
bi_lstm_y_pred = model.predict(X_test_vector)

bi_lstm_y_pred_inverse = one_hot_encoder.inverse_transform(bi_lstm_y_pred)

print(classification_report(y_test, bi_lstm_y_pred_inverse))

/Users/zhizhouhuang/Desktop/Labeled-Unreliable-News/venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)     │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_11 (Embedding)        │ (None, None, 100)      │     2,000,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ (None, None, 128)      │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_11     │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,088,940 (7.97 MB)

 Trainable params: 2,088,940 (7.97 MB)

 Non-trainable params: 0 (0.00 B)

/Users/zhizhouhuang/Desktop/Labeled-Unreliable-News/venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 24 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


94/94 ━━━━━━━━━━━━━━━━━━━━ 9s 92ms/step
              precision    recall  f1-score   support

           1       0.88      0.75      0.81       750
           2       0.73      0.58      0.65       750
           3       0.67      0.86      0.75       750
           4       0.85      0.91      0.88       750

    accuracy                           0.77      3000
   macro avg       0.78      0.77      0.77      3000
weighted avg       0.78      0.77      0.77      3000



In [48]:
import keras
from keras.layers import (
    Flatten,
    Embedding,
    GlobalAveragePooling1D,
    Bidirectional,
    LSTM,
    GlobalAveragePooling2D,
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, Adagrad
import tensorflow as tf

max_features = 100
epochs = 10
inputs = keras.Input(shape=(None,), dtype="int32")
embedding_layer = Embedding(num_tokens, embedding_dim, input_length=max_length)
embedding_layer.build()
embedding_layer.set_weights([embedding_matrix])
x = embedding_layer(inputs)

x = LSTM(64, return_sequences=True)(x)  # To add pooling here
x = GlobalAveragePooling1D()(x)  # Pooling layer
x = Dropout(0.5)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)


model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.001),
    metrics=[
        "accuracy",
        tf.keras.metrics.F1Score(
            average="macro", threshold=None, name="f1_score", dtype=None
        ),
    ],
)
model.summary()

checkpoint_filepath = "checkpoint_model_lstm_64_pooled.keras"

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_f1_score",
    mode="max",
    save_best_only=True,
)

model.fit(
    X_train_vector,
    y_train_labeled,
    epochs=epochs,
    validation_data=(X_test_vector, y_test_labeled),
    callbacks=[model_checkpoint_callback],
)

model.load_weights(checkpoint_filepath)
y_pred = model.predict(X_test_vector)

y_pred_inverse = one_hot_encoder.inverse_transform(y_pred)

print(classification_report(y_test, y_pred_inverse))

Model: "functional_35"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_20 (InputLayer)     │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_19 (Embedding)        │ (None, None, 100)      │     2,000,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_19 (LSTM)                  │ (None, None, 64)       │        42,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_18     │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_42 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_43 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,044,652 (7.80 MB)

 Trainable params: 2,044,652 (7.80 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 493s 322ms/step - accuracy: 0.5694 - f1_score: 0.5038 - loss: 0.9669 - val_accuracy: 0.6630 - val_f1_score: 0.6353 - val_loss: 1.4796
Epoch 2/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 500s 328ms/step - accuracy: 0.8771 - f1_score: 0.8691 - loss: 0.3560 - val_accuracy: 0.7197 - val_f1_score: 0.7047 - val_loss: 1.5740
Epoch 3/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 485s 318ms/step - accuracy: 0.9422 - f1_score: 0.9375 - loss: 0.1908 - val_accuracy: 0.7363 - val_f1_score: 0.7317 - val_loss: 1.6356
Epoch 4/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 499s 327ms/step - accuracy: 0.9715 - f1_score: 0.9692 - loss: 0.1007 - val_accuracy: 0.7303 - val_f1_score: 0.7226 - val_loss: 2.3860
Epoch 5/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 502s 329ms/step - accuracy: 0.9811 - f1_score: 0.9792 - loss: 0.0733 - val_accuracy: 0.7303 - val_f1_score: 0.7207 - val_loss: 2.1922
Epoch 6/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 476s 311ms/step - accuracy: 0.9899 - f1_score: 0.9890 - loss: 0.0419 - val_acc

In [49]:
import keras
from keras.layers import (
    Flatten,
    Embedding,
    GlobalAveragePooling1D,
    Bidirectional,
    LSTM,
    GlobalAveragePooling2D,
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, Adagrad
import tensorflow as tf

max_features = 100
epochs = 10
inputs = keras.Input(shape=(None,), dtype="int32")
embedding_layer = Embedding(num_tokens, embedding_dim, input_length=max_length)
embedding_layer.build()
embedding_layer.set_weights([embedding_matrix])
x = embedding_layer(inputs)

x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
# x = layers.Bidirectional(layers.LSTM(64, return ))(x)
x = GlobalAveragePooling1D()(x)  # Pooling layer
x = Dropout(0.5)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)


model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.001),
    metrics=[
        "accuracy",
        tf.keras.metrics.F1Score(
            average="macro", threshold=None, name="f1_score", dtype=None
        ),
    ],
)
model.summary()

checkpoint_filepath = "checkpoint_model_bi-lstm_64_pooled.keras"

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_f1_score",
    mode="max",
    save_best_only=True,
)

model.fit(
    X_train_vector,
    y_train_labeled,
    epochs=epochs,
    validation_data=(X_test_vector, y_test_labeled),
    callbacks=[model_checkpoint_callback],
)

model.load_weights(checkpoint_filepath)
bi_lstm_y_pred = model.predict(X_test_vector)

bi_lstm_y_pred_inverse = one_hot_encoder.inverse_transform(bi_lstm_y_pred)

print(classification_report(y_test, bi_lstm_y_pred_inverse))

/Users/zhizhouhuang/Desktop/Labeled-Unreliable-News/venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_37"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_21 (InputLayer)     │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_20 (Embedding)        │ (None, None, 100)      │     2,000,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ (None, None, 128)      │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_19     │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_44 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_45 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,088,940 (7.97 MB)

 Trainable params: 2,088,940 (7.97 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 533s 347ms/step - accuracy: 0.6106 - f1_score: 0.5308 - loss: 0.9117 - val_accuracy: 0.7057 - val_f1_score: 0.6946 - val_loss: 1.2696
Epoch 2/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 521s 341ms/step - accuracy: 0.9077 - f1_score: 0.9021 - loss: 0.2974 - val_accuracy: 0.6733 - val_f1_score: 0.6563 - val_loss: 2.1250
Epoch 3/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 524s 343ms/step - accuracy: 0.9593 - f1_score: 0.9578 - loss: 0.1498 - val_accuracy: 0.7480 - val_f1_score: 0.7399 - val_loss: 1.5921
Epoch 4/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 520s 341ms/step - accuracy: 0.9773 - f1_score: 0.9758 - loss: 0.0865 - val_accuracy: 0.7177 - val_f1_score: 0.6968 - val_loss: 2.4285
Epoch 5/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 523s 342ms/step - accuracy: 0.9870 - f1_score: 0.9860 - loss: 0.0507 - val_accuracy: 0.7827 - val_f1_score: 0.7808 - val_loss: 1.8063
Epoch 6/10
1527/1527 ━━━━━━━━━━━━━━━━━━━━ 521s 341ms/step - accuracy: 0.9915 - f1_score: 0.9909 - loss: 0.0332 - val_acc

In [58]:
import keras
from keras.layers import (
    Flatten,
    Embedding,
    GlobalAveragePooling1D,
    SimpleRNN,  # Changed from LSTM to SimpleRNN
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
import tensorflow as tf

max_features = 100
epochs = 10
inputs = keras.Input(shape=(None,), dtype="int32")
embedding_layer = Embedding(num_tokens, embedding_dim, input_length=max_length)
embedding_layer.build()
embedding_layer.set_weights([embedding_matrix])
x = embedding_layer(inputs)

x = SimpleRNN(64, return_sequences=True)(x)
x = GlobalAveragePooling1D()(x)  # Pooling layer
x = Dropout(0.5)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.001),
    metrics=[
        "accuracy",
        tf.keras.metrics.F1Score(
            average="macro", threshold=None, name="f1_score", dtype=None
        ),
    ],
)
model.summary()

checkpoint_filepath = "checkpoint_model_rnn_64_pooled.keras"

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_f1_score",
    mode="max",
    save_best_only=True,
)

# model.fit(
#     X_train_vector,
#     y_train_labeled,
#     epochs=epochs,
#     validation_data=(X_test_vector, y_test_labeled),
#     callbacks=[model_checkpoint_callback],
# )

model.load_weights(checkpoint_filepath)
y_pred = model.predict(X_test_vector)

rnn_y_pred_inverse = one_hot_encoder.inverse_transform(y_pred)

print(classification_report(y_test, rnn_y_pred_inverse))

import keras
from keras.layers import (
    Flatten,
    Embedding,
    GlobalAveragePooling1D,
    Bidirectional,
    LSTM,
    GlobalAveragePooling2D,
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, Adagrad
import tensorflow as tf

max_features = 100
epochs = 10
inputs = keras.Input(shape=(None,), dtype="int32")
embedding_layer = Embedding(num_tokens, embedding_dim, input_length=max_length)
embedding_layer.build()
embedding_layer.set_weights([embedding_matrix])
x = embedding_layer(inputs)

x = LSTM(64, return_sequences=True)(x)  # To add pooling here
x = GlobalAveragePooling1D()(x)  # Pooling layer
x = Dropout(0.5)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)


model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.001),
    metrics=[
        "accuracy",
        tf.keras.metrics.F1Score(
            average="macro", threshold=None, name="f1_score", dtype=None
        ),
    ],
)
model.summary()

checkpoint_filepath = "checkpoint_model_lstm_64_pooled.keras"

# model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     monitor="val_f1_score",
#     mode="max",
#     save_best_only=True,
# )

# model.fit(
#     X_train_vector,
#     y_train_labeled,
#     epochs=epochs,
#     validation_data=(X_test_vector, y_test_labeled),
#     callbacks=[model_checkpoint_callback],
# )

model.load_weights(checkpoint_filepath)
y_pred = model.predict(X_test_vector)

lstm_64_y_pred_inverse = one_hot_encoder.inverse_transform(y_pred)

print(classification_report(y_test, lstm_64_y_pred_inverse))

import keras
from keras.layers import (
    Flatten,
    Embedding,
    GlobalAveragePooling1D,
    Bidirectional,
    LSTM,
    GlobalAveragePooling2D,
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, Adagrad
import tensorflow as tf

max_features = 100
epochs = 10
inputs = keras.Input(shape=(None,), dtype="int32")
embedding_layer = Embedding(num_tokens, embedding_dim, input_length=max_length)
embedding_layer.build()
embedding_layer.set_weights([embedding_matrix])
x = embedding_layer(inputs)

x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
# x = layers.Bidirectional(layers.LSTM(64, return ))(x)
x = GlobalAveragePooling1D()(x)  # Pooling layer
x = Dropout(0.5)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)


model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.001),
    metrics=[
        "accuracy",
        tf.keras.metrics.F1Score(
            average="macro", threshold=None, name="f1_score", dtype=None
        ),
    ],
)
model.summary()

checkpoint_filepath = "checkpoint_model_bi-lstm_64_pooled.keras"

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_f1_score",
    mode="max",
    save_best_only=True,
)

# model.fit(
#     X_train_vector,
#     y_train_labeled,
#     epochs=epochs,
#     validation_data=(X_test_vector, y_test_labeled),
#     callbacks=[model_checkpoint_callback],
# )

model.load_weights(checkpoint_filepath)
bi_lstm_y_pred = model.predict(X_test_vector)

bi_lstm_y_pred_inverse = one_hot_encoder.inverse_transform(bi_lstm_y_pred)

print(classification_report(y_test, bi_lstm_y_pred_inverse))


# append the predictions to the test data
test["rnn_predicted"] = rnn_y_pred_inverse
test["lstm_predicted"] = lstm_64_y_pred_inverse
test["bi_lstm_predicted"] = bi_lstm_y_pred_inverse

print(test[:5])

/Users/zhizhouhuang/Desktop/Labeled-Unreliable-News/venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_45"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_25 (InputLayer)     │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_24 (Embedding)        │ (None, None, 100)      │     2,000,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, None, 64)       │        10,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_23     │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_52 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_53 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,012,972 (7.68 MB)

 Trainable params: 2,012,972 (7.68 MB)

...

 Total params: 2,088,940 (7.97 MB)

 Trainable params: 2,088,940 (7.97 MB)

 Non-trainable params: 0 (0.00 B)

/Users/zhizhouhuang/Desktop/Labeled-Unreliable-News/venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 24 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 102ms/step
              precision    recall  f1-score   support

           1       0.81      0.84      0.82       750
           2       0.82      0.54      0.65       750
           3       0.67      0.82      0.74       750
           4       0.84      0.90      0.87       750

    accuracy                           0.78      3000
   macro avg       0.78      0.78      0.77      3000
weighted avg       0.78      0.78      0.77      3000

   class                                               text  rnn_predicted  \
0      1  When so many actors seem content to churn out ...              2   
1      1   In what football insiders are calling an unex...              1   
2      1  In a freak accident following Game 3 of the N....              1   
3      1  North Koreas official news agency announced to...              4   
4      1  The former Alaska Governor Sarah Palin would b...              1   

   bi_lstm_predicted  lstm_predicted  
0              

In [59]:
print(test[:5])

   class                                               text  rnn_predicted  \
0      1  When so many actors seem content to churn out ...              2   
1      1   In what football insiders are calling an unex...              1   
2      1  In a freak accident following Game 3 of the N....              1   
3      1  North Koreas official news agency announced to...              4   
4      1  The former Alaska Governor Sarah Palin would b...              1   

   bi_lstm_predicted  lstm_predicted  
0                  1               1  
1                  1               1  
2                  1               1  
3                  4               4  
4                  1               1  


In [60]:
# write test to csv
test.to_csv("./test_predictions_for_comparison.csv", index=False)